In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# 目标
- 在本教程中，将学习如何把图像从一种颜色空间变换到另一种颜色空间，例如：$BGR \leftrightarrow Gray$或$BGR \leftrightarrow HSV$等等。
- 然后将实现一个从视频中提取特定颜色物体的应用。
- 我们将学习以下函数： **cv2.cvtColor()**，**cv2.inRange()**等。

# 1. 改变颜色空间
在OpenCV中有超过150种颜色空间变换的方法。下面介绍两种最常使用的变换：$BGR \leftrightarrow Gray$和$BGR \leftrightarrow HSV$。

可以通过调用**cv2.cvtColor(input_image, flag)**来进行颜色空间变换，**flag**用于指定变换的类型。

对于$BGR \leftrightarrow Gray$可以使用**cv2.COLOR_BGR2GRAY**，类似的对于$BGR \leftrightarrow HSV$可以使用**cv2.COLOR_BGR2HSV**。要获得所有支持的变换，可以执行以下代码：

In [3]:
import cv2
flags = [i for i in dir(cv2) if i.startswith('COLOR_')]
print(flags)

['COLOR_BAYER_BG2BGR', 'COLOR_BAYER_BG2BGR_EA', 'COLOR_BAYER_BG2BGR_VNG', 'COLOR_BAYER_BG2GRAY', 'COLOR_BAYER_BG2RGB', 'COLOR_BAYER_BG2RGB_EA', 'COLOR_BAYER_BG2RGB_VNG', 'COLOR_BAYER_GB2BGR', 'COLOR_BAYER_GB2BGR_EA', 'COLOR_BAYER_GB2BGR_VNG', 'COLOR_BAYER_GB2GRAY', 'COLOR_BAYER_GB2RGB', 'COLOR_BAYER_GB2RGB_EA', 'COLOR_BAYER_GB2RGB_VNG', 'COLOR_BAYER_GR2BGR', 'COLOR_BAYER_GR2BGR_EA', 'COLOR_BAYER_GR2BGR_VNG', 'COLOR_BAYER_GR2GRAY', 'COLOR_BAYER_GR2RGB', 'COLOR_BAYER_GR2RGB_EA', 'COLOR_BAYER_GR2RGB_VNG', 'COLOR_BAYER_RG2BGR', 'COLOR_BAYER_RG2BGR_EA', 'COLOR_BAYER_RG2BGR_VNG', 'COLOR_BAYER_RG2GRAY', 'COLOR_BAYER_RG2RGB', 'COLOR_BAYER_RG2RGB_EA', 'COLOR_BAYER_RG2RGB_VNG', 'COLOR_BGR2BGR555', 'COLOR_BGR2BGR565', 'COLOR_BGR2BGRA', 'COLOR_BGR2GRAY', 'COLOR_BGR2HLS', 'COLOR_BGR2HLS_FULL', 'COLOR_BGR2HSV', 'COLOR_BGR2HSV_FULL', 'COLOR_BGR2LAB', 'COLOR_BGR2LUV', 'COLOR_BGR2Lab', 'COLOR_BGR2Luv', 'COLOR_BGR2RGB', 'COLOR_BGR2RGBA', 'COLOR_BGR2XYZ', 'COLOR_BGR2YCR_CB', 'COLOR_BGR2YCrCb', 'COLOR_BGR

> **注意**  
> 对于HSV，色调（Hue）的范围是$[0, 179]$，饱和度（Saturation）和值（Value）的范围是$[0, 255]$。不同的软件使用不同的范围，因此要与其他软件比较OpenCV的值，要归一化这些范围。

# 2. 物体追踪
现在知道了如何把BGR图像转换为HSV图像，可以利用这一结果提取某种颜色的物体。在HSV中表示颜色比BGR颜色空间要容易，下面的例子要追踪一个蓝色的物体。下面是实现的方法：
- 从视频中提取每一帧；
- 将BGR图像转为HSV图像；
- 对HSV图像进行阈值操作，找到蓝色的区域；
- 提取出蓝色物体，然后对图像做任何想要的操作。

下面的代码给出了实现细节：

In [ ]:
import numpy as np
import cv2

cap = cv2.VideoCapture(0)

while True:
    # 获取一帧图像
    _, frame = cap.read()
    frame = cv2.resize(frame, (540, 360))
    
    # 从BGR转换到HSV
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # 定义HSV中蓝色的范围
    lower_blue = np.array([110, 50, 50])
    upper_blue = np.array([130, 255, 255])
#     lower_blue = np.array([7, 28, 50])
#     upper_blue = np.array([20, 255, 255])
    
    # 对HSV图像进行阈值操作，只保留蓝色
    mask = cv2.inRange(hsv, lower_blue, upper_blue)
    
    # 原始图像和自己进行带mask的按位与运算
    res = cv2.bitwise_and(frame, frame, mask=mask)
    
    cv2.imshow('frame', frame)
    cv2.imshow('mask', mask)
    cv2.imshow('res', res)
    k = cv2.waitKey(100) & 0xFF
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()

> **注意**  
> 图像中会存在一些噪声，后面的章节会介绍如何去除这些噪声。  
> 这是一个简单的物体追踪的方法，在学习了**contours**函数后，可以做诸如：定位物体中心来追踪物体、在摄像头前挥手来画图等有趣的操作。

# 3. 如何找到需要追踪的HSV值
这个问题在[StackOverFlow.com](http://statckoverflow.com)上非常常见，可以用**cv2.cvtColor()**函数来做到。直接把BGR的值传递给该函数，而不是整张图像。例如，要找到HSV中绿色的值，尝试执行下面的代码：

In [1]:
import cv2
import numpy as np
green = np.uint8([[[0, 255, 0]]])
hsv_green = cv2.cvtColor(green, cv2.COLOR_BGR2HSV)
print(hsv_green)

[[[ 60 255 255]]]


然后就可以用**[H-10, 100, 100]**和**[H+10, 255, 255]**分别作为下界和上界。除了这种方法，也可以用任何的图像编辑工具（例如GIMP）或者任意的在线工具获取这些值。

# 4. 练习
1. 尝试提取多种不同颜色的物体，例如同时提取红、绿、蓝三种物体。